In [2]:
from linkml_runtime.dumpers import json_dumper
#import python dataclasses generated from schema
from schema import Container, LOINC, Part, PartType
import pandas as pd
import numpy as np

#import data and load LPL271 sheet into a pandas dataframe
LPL = pd.read_excel('/Users/victoria/Documents/GitHub/comp-loinc/loinc_schema/CHEM_HIERARCHY_LPL_DATA.xlsx', 
                       sheet_name='LPL')

#Rename columns with blank spaces, underscores, and brackets since itertuples are column name sensitive
LPL_format = LPL.rename(columns={ 'PART_NUM' : 'PartCode', 'PART' : 'PartName', 
            'TYPE' : 'PartTypeCode', 'NAME' : 'PartTypeName',
           'LOINC_NUM' : 'LOINCCode', 'LONG_COMMON_NAME': 'LOINCName'})

#Assign unique id for each unique row
LPL_format['LOINCId'] =  LPL_format.groupby(['LOINCCode','LOINCName', 'PartCode', 'PartName', 
                                                   'PartTypeCode', 'PartTypeName'], 
                                                  sort=False).ngroup().apply('{:010}'.format)

#Format LOINCid column to LOINC(id_prefixes:https://loinc.org/):LOINC_0000000000
LPL_format['LOINCId'] = 'LOINC:LOINC_' + LPL_format['LOINCId'].astype(str)

#Create an empty list, ingest data using schema, append each row to empty list
loincdata=[]
for row in LPL_format.itertuples(index=False):
    data = LOINC(LOINCId = row.LOINCId, LOINCCode = row.LOINCCode, LOINCName = row.LOINCName, 
     has_part=[Part(PartCode = row.PartCode, PartName = row.PartName, 
     part_type=[PartType(PartTypeCode = row.PartTypeCode, PartTypeName = row.PartTypeName)])])
    
    loincdata.append(data)


container = Container(loincdata)

#write output as data.json
with open('CHEM_HIERARCHY_DATA.json', 'w') as f:
  f.write(json_dumper.dumps(container))

In [8]:
import graphviz as gv
import pandas as pd
Hierarchy271 = pd.read_excel('/Users/victoria/Documents/GitHub/comp-loinc/loinc_schema/CHEM_HIERARCHY_LPL_DATA.xlsx', 
                       sheet_name='Hierarchy')
Hierarchy271_format = Hierarchy271.rename(columns={ 'NODE_ID' : 'NodeID', 'PARENT_ID' : 'ParentID', 
            'FK_ID' : 'PartCode', 'PART' : 'PartName'})

dfObj1 = Hierarchy271_format.head(20000)
# dfObj1 = Hierarchy271_format.iloc[10000:20000]
g = gv.Digraph()

for row in dfObj1.itertuples(index=False):
    g.node(name=str(row.NodeID), label=row.PartName)

for row in dfObj1.itertuples(index=False):
    g.edge(str(row.NodeID), str(row.ParentID), label='is_a')

In [9]:
g.render()

'Digraph.gv.pdf'